In [0]:
%matplotlib inline
import os
import numpy as np
import pandas as pd
pd.set_option('display.max_colwidth', -1)


In [2]:
!pip install ktrain

     |████████████████████████████████| 92kB 2.8MB/s 
     |████████████████████████████████| 1.0MB 8.8MB/s 
     |████████████████████████████████| 245kB 26.5MB/s 
     |████████████████████████████████| 1.8MB 39.8MB/s 
  Created wheel for ktrain: filename=ktrain-0.7.2-cp36-none-any.whl size=113492 sha256=8ab0c0b87c44d727d148b642c1fa14c84a462e1729ab42484a16d3c1530e3ceb
  Stored in directory: /root/.cache/pip/wheels/ed/db/bd/fb2cb12563e8a7e5ef04f99cc9014bc0e509f1bb3834e6ee5d
  Created wheel for keras-bert: filename=keras_bert-0.80.0-cp36-none-any.whl size=37923 sha256=c915dfcd0c62dd205305d1128e09bddbe6f6cd041cb8b34c3793252de5d9bce4
  Stored in directory: /root/.cache/pip/wheels/63/dc/87/3260cb91f3aa32c0f85c5375429a30c8fd988bbb48f5ee21b0
  Created wheel for langdetect: filename=langdetect-1.0.7-cp36-none-any.whl size=993460 sha256=2f41a86deb87a143b60d4713e39985afa6c6f3fcd4fc05aa13f7d0322e8f3ba5
  Stored in directory: /root/.cache/pip/wheels/ec/0c/a9/1647275e7ef5014e7b83ff30105180e332867

In [3]:
import ktrain
ktrain.__version__

using Keras version: 2.2.4-tf


'0.7.2'

In [5]:
from sklearn.datasets import fetch_20newsgroups
remove = ('headers', 'footers', 'quotes')
newsgroups_train = fetch_20newsgroups(subset='train', remove=remove)
newsgroups_test = fetch_20newsgroups(subset='test', remove=remove)
texts = newsgroups_train.data +  newsgroups_test.data

In [6]:
tm = ktrain.text.get_topic_model(texts, n_features=10000)

n_topics automatically set to 97
lang: en
preprocessing texts...
fitting model...
iteration: 1 of max_iter: 5
iteration: 2 of max_iter: 5
iteration: 3 of max_iter: 5
iteration: 4 of max_iter: 5
iteration: 5 of max_iter: 5
done.


In [9]:
%%time
tm.build(texts, threshold=0.25)

done.
CPU times: user 14.3 s, sys: 7.71 s, total: 22 s
Wall time: 12.7 s


In [10]:
tm.print_topics()

topic 0 | tape adam tim case moved bag quote mass marked zionism
topic 1 | image jpeg images format programs tiff files jfif save lossless
topic 2 | alternative movie film static cycles films philips dynamic hou phi
topic 3 | hell humans poster frank reality kent gerard gant eternal bell
topic 4 | air phd chz kit cbc ups w-s rus w47 mot
topic 5 | dog math great figure poster couldn don trying rushdie fatwa
topic 6 | collaboration nazi fact end expression germany philly world certified moore
topic 7 | gif points scale postscript mirror plane rendering algorithm polygon rayshade
topic 8 | fonts font shell converted iii characters slight composite breaks compress
topic 9 | power station supply options option led light tank plastic wall
topic 10 | transmission rider bmw driver automatic shift gear japanese stick highway
topic 11 | tyre ezekiel ruler hernia appeared appointed supreme man land power
topic 12 | space nasa earth data launch surface solar moon mission planet
topic 13 | israel j

In [0]:
religious_topics = [36,40,74,60]

In [0]:
tm.train_scorer(topic_ids=religious_topics)

In [0]:
other_topics = [i for i in range(tm.n_topics) if i not in religious_topics]

In [0]:
other_texts = [d[0] for d in tm.get_docs(topic_ids=other_topics)]

In [0]:
other_scores = tm.score(other_texts)

In [0]:
other_preds = [int(score > 0) for score in other_scores]

In [0]:
data = sorted(list(zip(other_preds, other_scores, other_texts)), key=lambda item:item[1], reverse=True)

In [19]:
df = pd.DataFrame(data, columns=['Prediction', 'Score', 'Text'])
df.head()

,Prediction,Score,Text
0,1,0.257201,"The tongue that brings healing is a\n\ttree of life,\n\tbut a deceitful tongue crushes the\n\tspirit."
1,1,0.256935,"\n[FAQ and Darius' response deleted]\n\n\n\nI am myself an SDA and I am in total agreement with what Darius has to say. \nI also worship on Saturday to honor the Lord. Your mention of ""[esteeming] \nall days alike"" IMO has to do with the fast days observed by the Jews. But \nno matter how you interpret that passage, I do accept your worship on Sunday \nas being done in honor of the Lord, in contrast with what many of my fellow \nSDA believers may believe. To me, though, the bible overwhelmingly points \nto Saturday as the day to be kept in honor of creation and of God's \ndeliverance of the Israelites from Egyptian bondage. To those who would \nattempt to point out that my observance of Saturday is being legalistic, \nthis is simply not the case. Rather, keeping Saturday allows me a full day \nto rest and contemplate God's goodness and grace.\n\n\n\nThe idea was introduced to me once that the reason Paul wanted the \nCorinthians to lay aside money for the collection on the first day of the \nweek was because that was when they received their weekly wages. Paul \nwanted them to lay aside money for the collection as first priority, before \nspending their money on other things. I do not have any proof in front of \nme for this though, although it would explain why they would lay aside money \nin their homes instead of a meeting. \n\n\nIt would seem to me that you assume that the christians in the NT regularly \nworshipped on the first day. I assume that the christians in the NT \nregularly worshipped on the seventh day. But I agree with you that we only \nhave implications because the authors did assume the reader knew when worhip \nwas."
2,1,0.256664,"\n The dogma of the Assumption does not state whether or not Mary died a\nphysical death before being taken into Heaven. Catholics are free to\nbelieve what they wish, whether it be that she was taken still alive, or\nafter having died. I lean somewhat toward the latter myself."
3,1,0.252121,"\n\n\n\n\n\n\tAgain I find myself wanting to respond to a posting and having neither\nthe time nor the proper materials with me (you would think I would learn my\nlesson by now--but I'm trying to finish writing my Thesis and don't have tons\nof time. Anyway...)\n\n\tThe basis for our (the catholic church's) belief in the assumption of\nMary, body and soul, into heaven is that, to put it simply, the apostles \nand all the early generation Christians believed it. In fact, throughout their\nministry the apostles kept in close contact with Mary, and 11 of the 12 were\npresent when she died. Only Thomas was missing--when he arrived several days\nlater, he asked to be shown her body, and moved with pity, Peter and several of\nthe other apostles brought him to her tomb. When they arrived the seal was\nstill unbroken. They broke the seal, entered, and the body was missing. There\nwas no sign that anyone had entered, forcibly or otherwise, and everything else\nwas laid out exactly as it had been left. The apostles present all believed\nthat Mary was assumed into heaven--and the apostles TAUGHT this in their \npreaching (of course, this does not appear in any of the texts currently \nconsidered part of the bible, but it does appear in other writings left behind\nby several of them.) Basicaly, as an apostolic church (ie. founded by the\napostles), we believe that the teachings of the apostles, whether written down\nin the bible or written down in other sources, is true, providing that the\nauthenticity of those other sources can be confirmed. At least in the case of\nthe assumption of Mary, the authenticity is quite clear."
4,1,0.250008,"The traditions of the church hold that all the ""apostles"" (meaning the 11\nsurviving disciples, Matthias, Barnabas and Paul) were martyred, except for\nJohn. ""Tradition"" should be understood to read ""early churc

In [20]:
print('\t\tNumber of Predicted Inliners: %s' % sum(other_preds))

		Number of Predicted Inliners: 124


In [21]:
results = tm.search('war', case_sensitive=False)

In [22]:
print(results[0][0])


Don't be so sure.  Look what happened to Japanese citizens in the US during
World War II.  If you're prepared to say "Let's round these people up and
stick them in a concentration camp without trial", it's only a short step to
gassing them without trial.  After all, it seems that the Nazis originally
only intended to imprison the Jews; the Final Solution was dreamt up partly
because they couldn't afford to run the camps because of the devastation
caused by Goering's Total War.  Those who weren't gassed generally died of
malnutrition or disease.



In [0]:
doc_ids = [doc[1] for doc in results]
tm.train_scorer(doc_ids=doc_ids)

In [0]:
other_texts = [d[0] for d in tm.get_docs() if d[1] not in doc_ids]
other_scores = tm.score(other_texts)

In [0]:
other_preds = [int(score > 0) for score in other_scores]
data = sorted(list(zip(other_preds, other_scores, other_texts)), key=lambda item:item[1], reverse=True)

In [27]:
print('\t\tNumber of Predicted Inliners: %s' % sum(other_preds))

		Number of Predicted Inliners: 10061


In [28]:
df = pd.DataFrame(data, columns=['Prediction', 'Score', 'Text'])
df.head(3)

,Prediction,Score,Text
0,1,0.341649,"\n\nEd of course has never demonstrated remarkable knowlege of socialism, \nor any other political system come to that.\n\n\nThat leaves Germany, Japan and the UK as examples of a country where the\nright wing government is on the verge of collapse. Oh and of course the USA\nwhich just elected a socialist government :-)\n\n\nEtc, unfortunately you can't pin this on the left or the right, both are\nto blame. Both sides are equally deep into the corruption scandal. The only \nuntained party is the northern league which is a bunch of nationalist\nseparatists and the communist party which has collapsed.\n\n\n\nEd should take a look at the budget deficit Regan and Bush created together\nbefore he starts to make claims about europe collapsing based on the budget\ndeficits here. None of them are serious on the USA scale.\n\nAnd here in Europe we have zero interest in Ed-Ipser type freed thank you.\nWe do not want our countries to be run by a narrow elite of rich lawyers\nfor the benefit of the super wealthy. We are quite happy with social \ndemocracy and despite the fuss made in Time and Newsweek there is remarkably\nlittle being done to reverse the social welfare reforms brought in by\nsocialism.\n\nThe problem with socialism is that it started with the aims of free education\nand health care and provision of the welfare state. This has been achieved\nacross the whole of Europe, only the USA is struggling to catch up. The\nproblem for socialism is what to do now it has succeeded.\n\n\n\nEd starts to discus LA, presumably he thinks that it is in Europe. On\nthe other hand he most probably hasn't heard of a European city.\n\n\nRather the opposite. What is happening in Italy is that the communist party\nhas collapsed. This has meant that the grand coalition between right and\nleft wing parties to keep out the communists has also collapsed. The \nmagistrates have seized this opportunity to crack down hard on fraud and \ncorruption and have arrested half the politicians. The fact that the socialists\nare in charge this week is incidental, the right is into the corruption just\nas baddly.\n\nWhat looks likely to happen is the fringe parties are going to do much\nbetter in the next election. Most of the parliamentary deputies are going\nto get replaced and the parties are going to be forced to look to people\nwho are free of any hint of corruption. Look out for a parliament of\nPavarotti's and porn stars.\n"
1,1,0.337863,"Yarn of Cargo of Human Bones [1]\n\t\n\t\tCopyright, 1924, by the New York Times Company\n\t\t\tSpecial Cable to The New York Times\n\n PARIS, Dec 22, -- Marseilles is excited by a weird story of the arrival in\nthat port of a ship flying the British flag and named Zan carrying a\nmysterious cargo of 400 tons of human bones consigned to manufacturers there.\nThe bones are said to have been loaded at Mudania on the Sea of Marmora and\nto be the remains of the victims of massacres in Asia Minor. In view of the\nrumors circulating it is expected that an inquiry will be instigated.\n\n\t\t\t- - - Reference - - -\n\n[1] _New York Times_, December 23, 1924, page 3, column 2 (bottom)\n\n\t\t\t- - - - - - - - - - - -\n\nOn the 78th Commemorative Anniversary of the Turkish genocide of the Armenians,\nwe remember those whose only crime was to be Armenian in the shadow of an \nemerging Turkish proto-fascist state. In their names we demand justice.\n\nIn April 1915, the Turkish government began a systematically executed \nde-population of the eastern Anatolian homeland of the Armenians through a \ngenocidal extermination. This genocide was to insure that Turks exclusively\nruled over the geographic area today called the Republic of Turkey. The \nresult: 1.5 million murdered, 30 billion dollars of Armenian property stolen\nand plundered. This genocide ended nearly 3,000 years of Armenian civilization\non those lands. Today, the Turkish government continues to scrape clean any\nvestige of a prior Ar